In [ ]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.5 MB/s eta 0:00:00
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [ ]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [ ]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2023-12-31'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2023-12-31
Área de Interesse: Bioma Cerrado


In [ ]:
# ==============================================================================
# PASSO 2: CARREGAR ASSETS DE FOCOS DE CALOR
# ==============================================================================
print("Iniciando o carregamento dos assets de focos de calor...")

# 1. Definição caminho-base (pasta) onde os assets estão.
BASE_PATH = 'projects/the-byway-476116-n7/assets/'

# 2. Crie uma lista com os nomes de todos os seus assets anuais.
asset_names = [
    'bdqueimadas_2014-01-01_2014-12-31',
    'bdqueimadas_2015-01-01_2015-12-31',
    'bdqueimadas_2016-01-01_2016-12-31',
    'bdqueimadas_2017-01-01_2017-12-31',
    'bdqueimadas_2018-01-01_2018-12-31',
    'bdqueimadas_2019-01-01_2019-12-31',
    'bdqueimadas_2020-01-01_2020-12-31',
    'bdqueimadas_2021-01-01_2021-12-31',
    'bdqueimadas_2022-01-01_2022-12-31',
    'bdqueimadas_2023-01-01_2023-12-31'
]

# 3. Crie uma lista VAZIA para armazenar os objetos GEE
list_of_collections = []

# 4. Iteração pela lista de nomes para carregar cada asset
print(f"Encontrados {len(asset_names)} nomes de assets. Carregando do GEE...")

for name in asset_names:
    # Constrói o caminho completo do asset
    full_asset_path = BASE_PATH + name

    # Carrega o asset do GEE como uma FeatureCollection
    fc = ee.FeatureCollection(full_asset_path)

    # Adiciona a coleção carregada à nossa lista
    list_of_collections.append(fc)

print(f"Carregou {len(list_of_collections)} coleções de features com sucesso.")

# 5. Junção de todas as coleções em uma única e grande FeatureCollection
active_fires_collection = ee.FeatureCollection(list_of_collections).flatten()
print(active_fires_collection.first().getInfo())

print("\nAssets anuais de focos de calor mesclados com sucesso no servidor!")
print("O objeto 'active_fires_collection' está pronto.")

Iniciando o carregamento dos assets de focos de calor...
Encontrados 10 nomes de assets. Carregando do GEE...
Carregou 10 coleções de features com sucesso.
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-44.80399949627662, -11.890001217668136]}, 'id': '0_000000000000000001b7', 'properties': {'Bioma': 'Cerrado', 'DataHora': '2014/01/18 16:39:00', 'DiaSemChuv': 3, 'Estado': 'BAHIA', 'FRP': 8.8, 'Latitude': -11.89, 'Longitude': -44.804, 'Municipio': 'ANGICAL', 'Pais': 'Brasil', 'Precipitac': 1.6, 'RiscoFogo': 0.6, 'Satelite': 'AQUA_M-T'}}

Assets anuais de focos de calor mesclados com sucesso no servidor!
O objeto 'active_fires_collection' está pronto.


In [ ]:
# ==============================================================================
# PASSO 3: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- .4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print(land_cover.bandNames().getInfo())

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87624 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3630 imagens de LST (diárias).
Dados de elevação e declive carregados.
['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_

In [46]:
# ==============================================================================
# PASSO 4 (NOVO - CORRIGIDO v4): FUNÇÃO MESTRA DE ALINHAMENTO
# ==============================================================================
# Checa se as imagens (LST, NDVI, ERA5) existem (não são nulas)
# Antes de tentar executar o .sample() nelas.

print("Definindo a Função Mestra de Alinhamento (align_all_features) [CORRIGIDA v4]...")

# 1. Renomeia as imagens estáticas ANTES da função
elevation_renamed = elevation.rename('elevation_static')
slope_renamed = slope.rename('slope_static')
static_predictors = ee.Image.cat([elevation_renamed, slope_renamed])


def align_all_features(feature):

    # --- 1. PREPARAÇÃO (Datas e Valores Padrão) ---
    datahora_object = feature.get('DataHora')
    no_data = ee.Number(-9999)
    default_values = {
        'elevation_static': no_data, 'slope_static': no_data, 'lulc_class': no_data,
        'lst_daily_kelvin': no_data, 'ndvi_16day_prefire': no_data, 'temp_2m_C': no_data,
        'relative_humidity_percent': no_data, 'wind_u_ms': no_data,
        'wind_v_ms': no_data, 'wind_speed_ms': no_data, 'precipitation_hourly_mm': no_data
    }

    def process_with_date(date_obj):
        # --- 2. PREPARAÇÃO DAS DATAS ---
        date_string = ee.String(date_obj)
        date_only_string = date_string.slice(0, 10)
        year_string = date_string.slice(0, 4)
        formatted_date_string = date_only_string.replace('/', '-', 'g')
        fire_date = ee.Date(formatted_date_string)
        iso_string_step1 = date_string.replace('/', '-', 'g')
        iso_string_final = iso_string_step1.replace(' ', 'T')
        fire_datetime = ee.Date(iso_string_final)

        # --- 3. ALINHAMENTO ESTÁTICO (Sempre existe) ---
        static_sampled = static_predictors.sample(
            region=feature.geometry(), scale=TARGET_RESOLUTION, numPixels=1
        ).first()

        # --- 4. ALINHAMENTO ANUAL LULC (Sempre existe) ---
        lulc_band_name = ee.String('classification_').cat(year_string)
        lulc_image_this_year = land_cover.select(lulc_band_name)
        lulc_sampled = lulc_image_this_year.sample(
            region=feature.geometry(), scale=TARGET_RESOLUTION, numPixels=1
        ).first()

        # --- 5. ALINHAMENTO DIÁRIO LST (PODE SER NULO) ---
        lst_image_this_day = lst_collection.filterDate(fire_date).first()
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        lst_sampled = ee.Algorithms.If(
            lst_image_this_day, # Se a imagem existir...
            ee.Image(lst_image_this_day).sample(region=feature.geometry(), scale=1000, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulo
        )

        # --- 6. ALINHAMENTO QUINZENAL NDVI (PODE SER NULO) ---
        search_start_ndvi = fire_date.advance(-30, 'day')
        pre_fire_ndvi_image = ndvi_collection \
                                .filterDate(search_start_ndvi, fire_date) \
                                .sort('system:time_start', False).first()
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        ndvi_sampled = ee.Algorithms.If(
            pre_fire_ndvi_image, # Se a imagem existir...
            ee.Image(pre_fire_ndvi_image).sample(region=feature.geometry(), scale=1000, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulo
        )

        # --- 7. ALINHAMENTO HORÁRIO ERA5 (PODE SER NULO) ---
        search_start_era5 = fire_datetime.advance(-2, 'hour')
        era5_image_this_hour = era5_collection \
                                .filterDate(search_start_era5, fire_datetime) \
                                .sort('system:time_start', False).first()
        bands_to_sample = [
            'temperature_2m', 'dewpoint_temperature_2m', 'u_component_of_wind_10m',
            'v_component_of_wind_10m', 'total_precipitation_hourly'
        ]
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        era5_sampled = ee.Algorithms.If(
            era5_image_this_hour, # Se a imagem existir...
            ee.Image(era5_image_this_hour).select(bands_to_sample).sample(region=feature.geometry(), scale=11132, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulo
        )

        # --- 8. EXTRAÇÃO E CÁLCULOS (Agora com segurança) ---
        # (Todos os '..._sampled' podem ser nulos, por isso o 'If' é vital aqui)
        elev = ee.Algorithms.If(static_sampled, static_sampled.get('elevation_static'), no_data)
        slope = ee.Algorithms.If(static_sampled, static_sampled.get('slope_static'), no_data)
        lulc = ee.Algorithms.If(lulc_sampled, lulc_sampled.get(lulc_band_name), no_data)
        lst_k = ee.Algorithms.If(lst_sampled, ee.Number(ee.Feature(lst_sampled).get('LST_Day_1km')).multiply(0.02), no_data)
        ndvi = ee.Algorithms.If(ndvi_sampled, ee.Number(ee.Feature(ndvi_sampled).get('NDVI')).multiply(0.0001), no_data)

        # Converte o resultado de 'era5_sampled' (que pode ser nulo) para um ee.Feature
        # Isso permite que .get() funcione dentro do If
        era5_feature = ee.Feature(era5_sampled)

        temp_c = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('temperature_2m')).subtract(273.15), no_data)
        dew_c = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('dewpoint_temperature_2m')).subtract(273.15), no_data)
        wind_u = ee.Algorithms.If(era5_sampled, era5_feature.get('u_component_of_wind_10m'), no_data)
        wind_v = ee.Algorithms.If(era5_sampled, era5_feature.get('v_component_of_wind_10m'), no_data)
        precip_mm = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('total_precipitation_hourly')).multiply(1000), no_data)

        temp_c_num = ee.Number(temp_c)
        dew_c_num = ee.Number(dew_c)
        wind_u_num = ee.Number(wind_u)
        wind_v_num = ee.Number(wind_v)

        Ea = ee.Number(6.1094).multiply(((ee.Number(17.625).multiply(dew_c_num)).divide(dew_c_num.add(243.04))).exp())
        Es = ee.Number(6.1094).multiply(((ee.Number(17.625).multiply(temp_c_num)).divide(temp_c_num.add(243.04))).exp())
        rh = Ea.divide(Es).multiply(100)
        wind_speed = wind_u_num.pow(2).add(wind_v_num.pow(2)).sqrt()

        # --- 9. RETORNA O FEATURE FINAL E COMPLETO ---
        return feature.set({
            'elevation_static': elev, 'slope_static': slope, 'lulc_class': lulc,
            'lst_daily_kelvin': lst_k, 'ndvi_16day_prefire': ndvi, 'temp_2m_C': temp_c_num,
            'relative_humidity_percent': ee.Algorithms.If(rh.gt(0).And(rh.lte(100)), rh, no_data),
            'wind_u_ms': wind_u_num, 'wind_v_ms': wind_v_num, 'wind_speed_ms': wind_speed,
            'precipitation_hourly_mm': precip_mm
        })

    return ee.Algorithms.If(
        datahora_object,
        process_with_date(datahora_object),
        feature.set(default_values)
    )

# ==============================================================================
# APLICA A FUNÇÃO MESTRA
# ==============================================================================
print("Aplicando a Função Mestra a todos os pontos (definindo a 'receita')...")

dataset_final_positivo = active_fires_collection.map(align_all_features, True)

print("Receita final 'dataset_final_positivo' criada com sucesso.")

Definindo a Função Mestra de Alinhamento (align_all_features) [CORRIGIDA v4]...
Aplicando a Função Mestra a todos os pontos (definindo a 'receita')...
Receita final 'dataset_final_positivo' criada com sucesso.


In [48]:
# ==============================================================================
# PASSO 4.T: TESTE DA FUNÇÃO MESTRA (A Verificação)
# ==============================================================================
import pprint
print("Iniciando Teste da Função Mestra (com 10 pontos de 2014)...")

# (As variáveis BASE_PATH e asset_names devem existir das células anteriores)

try:
    # 1. Crie uma amostra de teste LEVE (10 pontos de 2014)
    asset_2014_path = BASE_PATH + asset_names[0]
    test_collection = ee.FeatureCollection(asset_2014_path).filter(ee.Filter.eq('Municipio','BRAS�LIA')).limit(10)

    # 2. Aplique a NOVA função 'align_all_features' (definida no Passo 4)
    test_result_master = test_collection.map(align_all_features)

    # 3. Chame o .getInfo() no RESULTADO do teste
    print("\nTeste Concluído. Inspecionando os 10 pontos de amostra:")
    pprint.pprint(test_result_master.getInfo())

    print("\n>>> SUCESSO! Todas as 'features' (Topografia, LULC, LST, NDVI, ERA5) devem estar visíveis.")

except Exception as e:
    print(f"\n>>> FALHA NO TESTE: {e}")

Iniciando Teste da Função Mestra (com 10 pontos de 2014)...

Teste Concluído. Inspecionando os 10 pontos de amostra:
{'columns': {'Bioma': 'String',
             'DataHora': 'String',
             'DiaSemChuv': 'Integer',
             'Estado': 'String',
             'FRP': 'Float',
             'Latitude': 'Float',
             'Longitude': 'Float',
             'Municipio': 'String',
             'Pais': 'String',
             'Precipitac': 'Float',
             'RiscoFogo': 'Float',
             'Satelite': 'String',
             'elevation_static': 'Object',
             'lst_daily_kelvin': 'Number',
             'lulc_class': 'Object',
             'ndvi_16day_prefire': 'Number',
             'precipitation_hourly_mm': 'Number',
             'relative_humidity_percent': 'Number',
             'slope_static': 'Object',
             'system:index': 'String',
             'temp_2m_C': 'Number',
             'wind_speed_ms': 'Number',
             'wind_u_ms': 'Object',
             '

In [ ]:
# ==============================================================================
# PASSO 5: EXPORTAÇÃO 1 (Amostras Positivas - Features Alinhadas)
# ==============================================================================
print("Iniciando o Passo 5: Preparação da Exportação 1...")
print("A variável 'dataset_final_positivo' (a 'receita' mestra) será enviada para exportação.")

# 1. Definir o nome do arquivo e a pasta no seu Google Drive
output_folder = 'ARTIGO_INCENDIOS_GEE'
output_filename = 'positive_samples_aligned_features'

# 2. Selecionar as colunas que queremos EXPORTAR
columns_to_export = [
    'DataHora', 'Latitude', 'Longitude', 'Municipio', 'Estado', 'Bioma', 'Satelite',
    'elevation_static', 'slope_static', 'lulc_class', 'lst_daily_kelvin',
    'ndvi_16day_prefire', 'temp_2m_C', 'relative_humidity_percent',
    'wind_u_ms', 'wind_v_ms', 'wind_speed_ms', 'precipitation_hourly_mm'
]

# 3. Criar a "Tarefa" (Task) de Exportação
print(f"Definindo a tarefa de exportação para: '{output_folder}/{output_filename}.csv'")

task = ee.batch.Export.table.toDrive(
    collection=dataset_final_positivo,        # A "receita" mestra do Passo 4
    description='Export_Positive_Aligned_v2_MasterFunc',
    folder=output_folder,
    fileNamePrefix=output_filename,
    fileFormat='CSV',
    selectors=columns_to_export
)

# 4. INICIAR a tarefa no servidor do GEE
task.start()

print("="*60)
print(f"TAREFA ENVIADA! O nome da tarefa é: 'Export_Positive_Aligned_v2_MasterFunc'")
print("="*60)
print("\n>>> INSTRUÇÕES CRÍTICAS (LEIA ABAIXO):")
print("1. Vá para o GEE Code Editor: https://code.earthengine.google.com/")
print("2. Na aba 'TASKS' (Tarefas), encontre 'Export_Positive_..._MasterFunc' (LARANJA).")
print("3. Clique em 'RUN' e confirme.")
print("="*60)

In [49]:
# ==============================================================================
# PASSO 6.1: CARREGAR DADOS-FONTE ESTÁTICOS [CORRIGIDO v3]
# ==============================================================================
print("Carregando dados-fonte estáticos...")

# 1. Topografia (SRTM - Global)
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
print("  Topografia (SRTM) carregada.")

# 2. MapBiomas LULC (Coleção 10)
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print("  Cobertura do Solo (MapBiomas) carregada.")

# ============================================================
# A CORREÇÃO REAL ESTÁ AQUI:
# ============================================================
# 3. Estradas (OpenStreetMap / Google)
#    Este é o dataset "padrão-ouro" de estradas globais, sempre disponível.
roads = ee.FeatureCollection('projects/the-byway-476116-n7/assets/EstradasSCA')
print("  Estradas do assets GEE carregadas.")
# ============================================================

print("Dados-fonte prontos.")

Carregando dados-fonte estáticos...
  Topografia (SRTM) carregada.
  Cobertura do Solo (MapBiomas) carregada.
  Estradas do assets GEE carregadas.
Dados-fonte prontos.


In [51]:
# ==============================================================================
# PASSO 6.2: ENGENHARIA DE FEATURES ESTÁTICAS [OTIMIZADO]
# ==============================================================================
print("Iniciando engenharia de features estáticas...")

# --- 1. Lógica do Aspect (Topografia) ---
print("  Calculando Aspect (Northness & Eastness)...")
aspect_deg = ee.Terrain.aspect(elevation)
aspect_rad = aspect_deg.multiply(3.1415926535 / 180)
aspect_northness = aspect_rad.cos().rename('aspect_northness')
aspect_eastness = aspect_rad.sin().rename('aspect_eastness')


# --- 2. Lógica da Distância (Infraestrutura) ---
urban_mask = land_cover.select('classification_2022').eq(24)
water_mask = land_cover.select('classification_2022').eq(33)
MAX_DISTANCE = 50000

print("  Calculando Distância para Água (MapBiomas)...")
distance_to_water = water_mask.Not().distance(ee.Kernel.euclidean(MAX_DISTANCE, 'meters')) \
    .rename('distance_to_water')

print("  Calculando Distância para Área Urbana (MapBiomas)...")
distance_to_urban = urban_mask.Not().distance(ee.Kernel.euclidean(MAX_DISTANCE, 'meters')) \
    .rename('distance_to_urban')

# ============================================================
# OTIMIZAÇÃO DAS ESTRADAS
# ============================================================
print("  Calculando Distância para Estradas (OSM/Google)...")

# 1. Filtra a coleção global de estradas SOMENTE para a nossa AOI
roads_no_cerrado = roads.filterBounds(cerrado_aoi)

# 2. Agora, "queima" (paint) apenas as estradas do Cerrado.
roads_image = ee.Image().byte().paint(roads_no_cerrado, 1)
# ============================================================

distance_to_roads = roads_image.Not().distance(ee.Kernel.euclidean(MAX_DISTANCE, 'meters')) \
    .rename('distance_to_roads')

print("Engenharia de features concluída.")


# --- 3. Combinação ---
all_static_predictors = ee.Image.cat([
    aspect_northness,
    aspect_eastness,
    distance_to_water,
    distance_to_urban,
    distance_to_roads
])

print("Todas as features estáticas foram combinadas.")

Iniciando engenharia de features estáticas...
  Calculando Aspect (Northness & Eastness)...
  Calculando Distância para Água (MapBiomas)...
  Calculando Distância para Área Urbana (MapBiomas)...
  Calculando Distância para Estradas (OSM/Google)...
Engenharia de features concluída.
Todas as features estáticas foram combinadas.


In [52]:
# ==============================================================================
# PASSO 6.3 (SIMPLIFICADO): AMOSTRAGEM E EXPORTAÇÃO 2
# ==============================================================================
print("Iniciando amostragem das features estáticas...")

static_features_dataset = all_static_predictors.sampleRegions(
    collection=active_fires_collection,
    properties=['DataHora', 'Latitude', 'Longitude'],
    scale=TARGET_RESOLUTION,
    geometries=True
)

print("Amostragem concluída. Definindo a Tarefa de Exportação 2...")

output_folder = 'ARTIGO_INCENDIOS_GEE_Aspect_Roads'
output_filename = 'positive_samples_static_features'

task_static = ee.batch.Export.table.toDrive(
    collection=static_features_dataset,
    description='Export_Positive_Static_v2_Simplified', # Novo nome de tarefa
    folder=output_folder,
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# 3. INICIAR a tarefa
task_static.start()

print("="*60)
print(f"TAREFA ENVIADA! O nome da tarefa é: 'Export_Positive_Static_v2_Simplified'")
print("="*60)
print(">>> INSTRUÇÕES: Vá para a aba 'TASKS' no GEE e clique em 'RUN'.")
print("   Esta versão (sem estradas) é mais limpa e cientificamente robusta.")
print("="*60)

Iniciando amostragem das features estáticas...
Amostragem concluída. Definindo a Tarefa de Exportação 2...
TAREFA ENVIADA! O nome da tarefa é: 'Export_Positive_Static_v2_Simplified'
>>> INSTRUÇÕES: Vá para a aba 'TASKS' no GEE e clique em 'RUN'.
   Esta versão (sem estradas) é mais limpa e cientificamente robusta.
